### Data Collection ##

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# top 10개 회사 리뷰 추출

samsung = 'https://www.teamblind.com/kr/company/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90/' # 리뷰 갯수: 13,111
kakao = 'https://www.teamblind.com/kr/company/%EC%B9%B4%EC%B9%B4%EC%98%A4/' # 리뷰 갯수: 2,304
naver = 'https://www.teamblind.com/kr/company/NAVER/' # 리뷰 갯수: 3,178
hyundai ='https://www.teamblind.com/kr/company/%ED%98%84%EB%8C%80%EC%9E%90%EB%8F%99%EC%B0%A8/' # 리뷰 갯수: 6,713
amore = 'https://www.teamblind.com/kr/company/%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD/' # 리뷰 갯수: 1,704
cjenm = 'https://www.teamblind.com/kr/company/CJ%20ENM/' # 리뷰 갯수: 1,442
sk = 'https://www.teamblind.com/kr/company/SK%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4/' # 리뷰 갯수: 4,826
samsungcnt = 'https://www.teamblind.com/kr/company/%EC%82%BC%EC%84%B1%EB%AC%BC%EC%82%B0/' # 리뷰 갯수: 1,217
koreanair= 'https://www.teamblind.com/kr/company/%EB%8C%80%ED%95%9C%ED%95%AD%EA%B3%B5/' # 리뷰 갯수: 1,179
cj_food='https://www.teamblind.com/kr/company/CJ%EC%A0%9C%EC%9D%BC%EC%A0%9C%EB%8B%B9/' # 리뷰 갯수: 1,829

In [ ]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}

def review_title(name, company_url, num_review):
  import pandas as pd
  info, dates, ratings, titles = [], [], [], []
  total_pages = (num_review // 30) + 1 # 한 페이지당 30개 리뷰 존재

  for page in range(1, total_pages+1):
    r = requests.get(company_url + f'reviews?page={page}', headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    review = soup.find_all('div', attrs = {'class': 'review_item'})

    for item in review:
      ppl_info = item.find('div', attrs={'class' : 'auth'}).get_text().strip()
      date = ppl_info.split(' ')[-1] # date
      rating = item.find('strong', attrs={'class' : 'num'}).get_text() # rating
      title = item.find('h3', attrs={'class': 'rvtit'}).get_text() # title

      info.append(ppl_info)
      dates.append(date)
      ratings.append(rating)
      titles.append(title)

  output =  {'info': info, 'date': dates, 'rating': ratings, 'title': titles}
  result = pd.DataFrame(output)

  result.to_csv(f'./[1]data_collection/blind_{name}.csv', encoding='utf8', index=False)

In [ ]:
samsung = review_title('samsung', samsung, 13111) # 437
amore_df = review_title('amore', amore, 1704)
kakao_df = review_title('kakao', kakao, 2304)
hyundai_df = review_title('hyundai', hyundai, 6713)
cjenm_df = review_title('cjenm', cjenm, 1442)
sk_df = review_title('sk', sk, 4826)
samsungcnt_df = review_title('samsungcnt', samsungcnt, 1217)
koreanair_df = review_title('koreanair', koreanair, 1179)
cj_food_df = review_title('cj_food', cj_food, 1829)

In [ ]:
# Other companies

corp = pd.read_csv('data_collection_corp_list.csv', encoding='utf8') # 시가총액 기준  탑 50 회사 리스트
corp = corp[corp['url'] != 'o']
corp['review_num'] = corp['review_num'].astype(int)
corp['url'] = corp['url'].apply(lambda x:x[:-7])
corp.url.iloc[1] # 예시

'https://www.teamblind.com/kr/company/SK%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4/'

In [ ]:
for i in corp.index:
  review_title(corp.loc[i, '이름'], corp.loc[i, 'url'], corp.loc[i, 'review_num'])